In [ ]:
import pandas as pd
train=pd.read_csv('/kaggle/input/playground-series-s3e17/train.csv')
train.head()

In [ ]:
test=pd.read_csv('/kaggle/input/playground-series-s3e17/test.csv')
test.head()

In [ ]:
train.nunique()

In [ ]:
train.info()

In [ ]:
train=train.drop(['id'],axis=1)
train.head()

In [ ]:
test=test.drop(['id'],axis=1)
test.head()

In [ ]:
list(set(train['Product ID'].unique().tolist())^set(test['Product ID'].unique().tolist()))[:5]

In [ ]:
train['Product ID'].value_counts()

In [ ]:
train=train.drop(['Product ID'],axis=1)
train.head()

In [ ]:
test=test.drop(['Product ID'],axis=1)
test.head()

In [ ]:
round(train['Machine failure'].value_counts()*100/len(train),2)

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('object')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
import numpy as np
train = reduce_mem_usage(train)
train.info()

In [ ]:
test = reduce_mem_usage(test)
test.info()

In [ ]:
train.duplicated().value_counts()

In [ ]:
test.duplicated().value_counts()

In [ ]:
train[train.duplicated()].head()

In [ ]:
test[test.duplicated()].head()

In [ ]:
print(train.shape)
train.drop_duplicates(inplace=True,keep='last')
train.shape

In [ ]:
import re
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
train.head()

In [ ]:
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test.head()

In [ ]:
!pip install flaml
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
from flaml import AutoML
automl = AutoML()

In [ ]:
y = train.pop('Machinefailure')
X = train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,shuffle=True, stratify=y)

In [ ]:
automl.fit(X_train, y_train, task="classification",metric='accuracy',time_budget=3600*4)

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, automl.predict(X_train)))

In [ ]:
print(classification_report(y_test, automl.predict(X_test)))

In [ ]:
y_pred = automl.predict_proba(test)[:,1]
y_pred[:5]

In [ ]:
df = pd.DataFrame(y_pred,columns=['Machine failure'])
df.head()

In [ ]:
sol=pd.read_csv('/kaggle/input/playground-series-s3e17/sample_submission.csv')
sol.head()

In [ ]:
sol['Machine failure']=df['Machine failure']
sol.head()

In [ ]:
sol.to_csv('./accuracy.csv',index=False)